Por ser uma linguagem funcional, Scala encoraja o uso de *funções puras* e *estruturas imutáveis*. No entanto, principalmente quando se interage com serviços externos, é difícil garantir que uma função sempre retorne o mesmo valor quando chamada com os mesmos argumentos. Uma maneira de manter as funções puras mesmo com a presença de *efeitos colaterais* na operação da função é encapsular o retorno da função em um tipo que represente esse efeito colateral - esses tipos que representam efeitos colaterais são chamados de ***mônadas***.

## Primeira mônada: Option

In [ ]:
val aMap = Map("one" -> 1, "two" -> 2)

aMap("one")

In [ ]:
aMap("three")  // Lança uma exceção, não é uma função pura

A mônada **Option** representa o efeito colateral da *possível ausência de um valor*. Ela permite, através dos métodos *isDefined* e *isEmpty*, checar se o valor está presente ou não e obter o valor, quando presente, através do método *get*.

Um **Option** pode assumir dois tipos: 
* Um *Some* quando o valor está presente;
* Um *None* quando o valor está ausente;

Esses tipos podem ser usadoss para se retornar um **Option** em um método:

In [ ]:
val valueForTwo = aMap.get("two")
val valueForThree = aMap.get("three")

if (valueForTwo.isDefined)
    println(valueForTwo.get)
else
    println("No mapped value for 'two'")

if (valueForThree.isDefined)
    println(valueForThree.get)
else
    println("No mapped value for 'three'")

In [ ]:
def iCanReturnAnOption(n: Int): Option[Int] = {
    if (n < 3)
        Some(n * 2)
    else
        None
}

iCanReturnAnOption(2)
iCanReturnAnOption(5)

## Outra mônada: Future

In [ ]:
val randomGenerator = scala.util.Random

def randomDelay() = {
    val waitingTime = randomGenerator.nextInt(5)
    Thread.sleep(waitingTime * 1000)
    
    waitingTime
}

In [ ]:
println(randomDelay())
println(randomDelay())

A mônada **Future** permite trabalhar-se com o possível resultado de um método quando existe um atraso entre o momento em que o método retorna e o momento em que o resultado dele torna-se disponível. Ela costuma ser usada como tipo de retorno em métodos que consultam serviços externos a fim de representar o tempo da consulta a esse serviço.
**Future** pode assumir dois tipos:
* *Success* quando, ao terminar de processar o retorno, o método retornar um valor com sucesso;
* *Failure* quando, ao terminar de processar o retorno, o método declara uma falha no processamento;

É possível fazer com que um método retorne um **Future** colocando um bloco *Future {}* em volta do corpo do método - valores retornados normalmente serão encapsulados em um *Success* e exceções serão retornadas como *Failures*.

In [ ]:
import scala.concurrent.{Await, Future}
import scala.concurrent.ExecutionContext.Implicits.global

def asyncDelay(shouldFail: Boolean): Future[Int] = Future {
    if (shouldFail)
        throw new Exception("Falhou :/")
    else {
        val waitingTime = 5
        Thread.sleep(waitingTime * 1000)
    
        waitingTime
    }
}

In [ ]:
import scala.util.{Success, Failure}

val firstAttempt = asyncDelay(false)
val secondAttempt = asyncDelay(true)

firstAttempt.onComplete {
    case Success(result) => println(result)
    case Failure(cause) => println(cause)
}
secondAttempt.onComplete {
    case Success(result) => println(result)
    case Failure(cause) => println(cause)
}

In [ ]:
import scala.concurrent.duration._

val thirdAttempt = asyncDelay(false)
Await.result(fourthAttempt, 6 seconds)

In [ ]:
val fourthAttempt = asyncDelay(false)
Await.result(thirdAttempt, 2 seconds)

## Mônadas, operadores e for

### Operadores

Um processamento comum feito sobre resultados com efeitos colaterais é tentar usar o valor desse resultado antes de se saber se ele está disponível ou se houve uma falha:
* Quando se tem um resultado **Option**, deseja-se ignorá-lo quando o resultado é *None*;
* Quando se tem um resultado **Future**, deseja-se ignorá-lo quando o resultado é *Failure*;

Já vimos anteriormente que pode-se usar if/else para isso, mas uma outra maneira é o operador *map*:

In [ ]:
val someOption: Option[Int] = Some(1)
val NoneOption: Option[Int] = None

val firstResult = someOption.map(_ * 2)
val secondResult = NoneOption.map(_ * 2) // "map" continua retornando None caso o valor de entrada já seja None

Uma variação do cenário acima é quando se tem um conjunto de resultados com efeitos colaterais e deseja-se remover todos os resultados de falha antes de se aplicar uma transformação. Isso pode ser obtido com o uso de *filter* seguido de *map*:

In [ ]:
val someResults = Seq(Some(1), Some(2), None, None, Some(5))

someResults.filter(_.isDefined).map(_.get * 2)

Uma maneira mais concisa e equivalente de se fazer isso é usando o operador *flatMap* - esse operador vai aplicar uma transformação a cada resultado do conjunto e então remover os resultados com efeito colateral de falha:

In [ ]:
someResults.flatMap(o => o.map(_ * 2))

### Uso do laço for com mônadas

Quando se tem resultados individuais com efeitos colaterais, é comum ter que se combiná-los para gerar um novo resultado - isso exige que se cheque se esses resultados individuais são falhas antes de combiná-los:

In [ ]:
if(firstResult.isDefined && secondResult.isDefined)
    println("Resultado geral é " + firstResult.get + secondResult.get)
else 
    println("Algum resultado falhou")

Uma maneira mais simples de se checar resultados individuais e combiná-los em um novo resultado em caso de sucesso é usando um laço *for* em Scala:

In [ ]:
val combinedResult = for {
    firstResultValue <- firstResult
    secondResultValue <- secondResult
} yield firstResultValue + secondResultValue

In [ ]:
val thirdResult = Some(3)

val combinedResult = for {
    firstResultValue <- firstResult
    thirdResultValue <- thirdResult
} yield firstResultValue + thirdResultValue

if(combinedResult.isDefined)
    println("Resultado geral é " + combinedResult.get)
else
    println("Algum resultado falhou")

O laço *for* permite extrair os valores encapsulados em um efeito colateral e usá-los em alguma expressão para formar um novo resultado ***encapsulado naquele mesmo tipo de efeito colateral***, retornando imediatamente um resultado de falha caso pelo menos um dos resultados de entrada seja uma falha.

Essa estratégia permite que se combine resultados com efeitos colaterais sem precisar checar se eles possuem algum valor e postergar a checagem de falhas apenas para o momento em que isso for estritamente necessário.